In [1]:
## Ensure the correct working directory

import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime
from IPython.display import clear_output

In [3]:
from workflow.pipeline import culture, ephys, probe

[2025-06-12 16:15:22,966][INFO]: Connecting judewerth@db.datajoint.com:3306
[2025-06-12 16:15:23,828][INFO]: Connected judewerth@db.datajoint.com:3306


In [ ]:
# Remove Sessions during times with no data
no_data = "BETWEEN '2023-07-13 1:59:00' AND '2023-07-13 11:54:00'"
s = f"start_time {no_data} OR end_time {no_data}"

delete_sessions = ephys.EphysSession & s
delete_sessions

In [ ]:
keys = delete_sessions.fetch("KEY")

for key in keys:
    print(f"delete times: 1:59:00-11:55:00")
    print(f"key times: {key['start_time']}, {key['end_time']}")

    info_query = ephys.EphysSessionInfo & key
    if len(info_query) > 0:
        print(f"{len(info_query)} Info Sessions Found")

        info_keys = info_query.fetch("KEY")
        (ephys.EphysSessionInfo & info_keys).delete()

    probe_query = (ephys.EphysSessionProbe & key)
    if len(probe_query) > 0:
        print(f"{len(probe_query)} EphysProbe Sessions Found")

        probe_keys = probe_query.fetch("KEY")
        (ephys.EphysSessionProbe & probe_keys).delete() 

    cluster_query = ephys.ClusteringTask & key
    if len(cluster_query) > 0:
        print(f"{len(cluster_query)} Clustering Tasks Found")

        cluster_keys = cluster_query.fetch("KEY")
        (ephys.ClusteringTask & cluster_keys).delete()

    lfp_query = ephys.LFP & key
    if len(lfp_query) > 0:
        print(f"{len(lfp_query)} LFP Sessions Found")

        lfp_keys = lfp_query.fetch("KEY")
        (ephys.LFP & lfp_keys).delete()

    print("Child tables deleted: Now delete the EphysSession")
    (ephys.EphysSession & key).delete()

    clear_output()

In [5]:
# if you get an error it is likely due to child tables having the key
# find them with
ephys.EphysSession.children()

['`utah_organoids_ephys`.`_ephys_session_info`',
 '`utah_organoids_ephys`.`_l_f_p`',
 '`utah_organoids_ephys`.`clustering_task`',
 '`utah_organoids_ephys`.`ephys_session_probe`']

IntegrityError: Cannot add or update a child row: a foreign key constraint fails (`utah_organoids_mua`.`__m_u_a_spikes__channel`, CONSTRAINT `__m_u_a_spikes__channel_ibfk_1` FOREIGN KEY (`organoid_id`, `experiment_start_time`, `start_time`, `threshold_uv`) REFERENCES `__m_u)

Traceback (most recent call last): 
File "/opt/conda/lib/python3.10/site-packages/datajoint/autopopulate.py", line 335, in _populate1 make(dict(key), **(make_kwargs or {})) 
File "/opt/conda/lib/python3.10/site-packages/workflow/pipeline/mua.py", line 170, in make self.Channel.insert1( File "/opt/conda/lib/python3.10/site-packages/datajoint/table.py", line 349, in insert1 self.insert((row,), **kwargs) 
File "/opt/conda/lib/python3.10/site-packages/datajoint/table.py", line 453, in insert self.connection.query( File "/opt/conda/lib/python3.10/site-packages/datajoint/connection.py", line 350, in query self._execute_query(cursor, query, args, suppress_warnings) 
File "/opt/conda/lib/python3.10/site-packages/datajoint/connection.py", line 306, in _execute_query raise translate_query_error(err, query) datajoint.errors.
IntegrityError: Cannot add or update a child row: a foreign key constraint fails (`utah_organoids_mua`.`__m_u_a_spikes__channel`, CONSTRAINT `__m_u_a_spikes__channel_ibfk_1` FOREIGN KEY (`organoid_id`, `experiment_start_time`, `start_time`, `threshold_uv`) REFERENCES `__m_u)
